In [28]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=8):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    imshape = img.shape

    left_slope = 0
    right_slope = 0
    right_x = 0
    left_x = 0
    
    longest_left = 0
    longest_right = 0
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (y2-y1)/(x2-x1)
            dist = math.sqrt((x2-x1)**2 + (y2-y1)**2)
            if slope < 0:
                if(dist > longest_left):
                    longest_left = dist 
                    left_slope = slope
                    left_x = (x2 + x1)/2
                    left_y = (y2+y1)/2
                    
            else:
                if(dist > longest_right):
                    longest_right = dist 
                    right_slope = slope
                    right_x = (x2 + x1)/2
                    right_y = (y2+y1)/2

    top_y = 330
    bottom_y = imshape[0]
    ys = (top_y, bottom_y)
    
    
    right_x_cords = get_x(right_x, right_y, right_slope, ys)
    top_right_x = right_x_cords[0]
    bottom_right_x = right_x_cords[1]
    
    left_x_cords = get_x(left_x, left_y, left_slope, ys)
    top_left_x = left_x_cords[0]
    bottom_left_x = left_x_cords[1]
    
    cv2.line(img, (int(top_left_x), top_y), (int(bottom_left_x), bottom_y), color, thickness)
    cv2.line(img, (int(top_right_x), top_y), (int(bottom_right_x), bottom_y), color, thickness)
    
    #cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
def draw_all_lines(img, lines, color=[255, 0, 0], thickness=6):
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)
    
def get_x(x, y, slope, ys):
    
    top_y = ys[0]
    bottom_y = ys[1]
    avg_y = (bottom_y + top_y)/2
    
    b = y - (slope*x)
    top_x = (top_y - b) / slope
    
    bottom_x = (bottom_y-b) / slope
    
    return (top_x, bottom_x)
    



def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    #draw_all_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    #reading in an image
    #image = mpimg.imread('test_images/WhiteCarLaneSwitch.jpg')
    #printing out some stats and plotting
    #print('This image is:', type(image), 'with dimesions:', image.shape)
    #plt.imshow(image)  
    gray = grayscale(image)
    #plt.imshow(gray, cmap='gray') #grayscaled image

    #apply Gaussian smoothing
    blur_gray = gaussian_blur(gray,5)
    #plt.imshow(blur_gray)

    # Define our parameters for Canny and apply
    edges = canny(blur_gray, 75, 225)
    #plt.imshow(edges, cmap='Greys_r')

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 330), (510, 330), (imshape[1],imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    #plt.imshow(masked_edges, cmap='Greys_r')

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    line_image = hough_lines(masked_edges, 1, np.pi/180, 35, 20, 8) 
    #plt.imshow(line_image)

    # Draw the lines on the original image
    original_with_lines = weighted_img(line_image, image, .8,1.0)
    #plt.imshow(original_with_lines)
    return original_with_lines


white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

#yellow_output = 'yellow.mp4'
#clip2 = VideoFileClip('solidYellowLeft.mp4')
#yellow_clip = clip2.fl_image(process_image)
#%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
 <source src="{0}">
</video>
""".format(white_output))



[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4





  0%|          | 0/222 [00:00<?, ?it/s]


  4%|▍         | 9/222 [00:00<00:02, 84.96it/s]


  9%|▊         | 19/222 [00:00<00:02, 87.87it/s]


 13%|█▎        | 29/222 [00:00<00:02, 90.49it/s]


 18%|█▊        | 39/222 [00:00<00:02, 91.15it/s]


 21%|██        | 47/222 [00:00<00:02, 78.65it/s]


 24%|██▍       | 54/222 [00:00<00:02, 74.33it/s]


 27%|██▋       | 61/222 [00:00<00:02, 64.67it/s]


 31%|███       | 68/222 [00:00<00:02, 60.63it/s]


 33%|███▎      | 74/222 [00:01<00:02, 58.97it/s]


 36%|███▌      | 80/222 [00:01<00:02, 56.45it/s]


 39%|███▊      | 86/222 [00:01<00:02, 51.29it/s]


 41%|████▏     | 92/222 [00:01<00:02, 46.20it/s]


 44%|████▎     | 97/222 [00:01<00:02, 47.02it/s]


 46%|████▋     | 103/222 [00:01<00:02, 49.14it/s]


 49%|████▉     | 109/222 [00:01<00:02, 49.83it/s]


 52%|█████▏    | 115/222 [00:01<00:02, 51.03it/s]


 55%|█████▍    | 121/222 [00:02<00:01, 51.91it/s]


 57%|█████▋    | 127/222 [00:02<00:01, 53.13it/s]


 60%|█████▉    | 133/222 [00:02<0

[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 3.21 s, sys: 919 ms, total: 4.13 s
Wall time: 4.65 s


The pipeline is very similar to that presented in Lesson 1:
    - Gray scale the image 
    - apply gaussian smoothing 
    - Canny edge detection
    - Mask the image to only consider the trapizoidal region where the road is most likely to appear
    - Hough on edge image
    - extrapolate lane line position and slope from largest detected line segment for both left and right lane lines (see discussion below)
    - apply color and transparency to lines


Extrapolation of Lane Lines:
My first attempt was to average the x-position and slope of all "left lines" and "right lines" (determined by positive or negative slope).  Unfortunately, because of the reflectors on the road, both the position averages and the slope averages were heavily skewed and resulted in very jittery lanes from the "noise" of the reflectors and other arbitrary lines seen on the road. 

My final algorithm extrapolates the lane line from the largest detected line segment on each side. Both the average lane line position and lane line slope is taken from the largest line segment detected. This approach is good in that it is difficult to skew the slope or position of the resulting lane line. The algorithm would perform poorly if the road was on a sharp curve, or if some object / erroneous line on the road fooled the line detector. 
